# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: black; padding: 10px; border-radius: 5px; margin-bottom: 20px'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format

# Load the model and tokenizer
# model_name = "HuggingFaceTB/SmolLM2-135M"
# model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to("cuda")
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# # Set up the chat format
# model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


c:\Users\kevor\anaconda3\envs\smol_course\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

In [ ]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 If your dataset is not in a 'chatml' format, you will need to process it. Refer to the [module](../chat_templates.md)
def process_dataset(sample):
    # Apply the chat template and store the result
    chat_formatted = tokenizer.apply_chat_template(sample['messages'], tokenize=False, add_generation_prompt=True)
    # Return a dictionary with the updated key-value pair(s)
    sample['messages'] = chat_formatted
    return sample

# Apply the processing function to the dataset
ds2 = ds.map(process_dataset)

In [ ]:
ds['train'][0]

In [ ]:
print(ds2["train"].column_names)
print(ds2["train"][0])


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    # use_mps_device=True,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./sft_output")

{'eval_loss': 1.0352314710617065, 'eval_runtime': 0.8442, 'eval_samples_per_second': 140.956, 'eval_steps_per_second': 17.768, 'epoch': 1.42}


{'loss': 0.6984, 'grad_norm': 1.7829028367996216, 'learning_rate': 9.5e-06, 'epoch': 1.43}


{'loss': 0.698, 'grad_norm': 1.8063157796859741, 'learning_rate': 9e-06, 'epoch': 1.45}


{'loss': 0.6922, 'grad_norm': 1.7154245376586914, 'learning_rate': 8.500000000000002e-06, 'epoch': 1.47}


{'loss': 0.7389, 'grad_norm': 1.844586730003357, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.49}


...

{'loss': 0.7163, 'grad_norm': 1.6691867113113403, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}


{'loss': 0.6709, 'grad_norm': 1.6968684196472168, 'learning_rate': 0.0, 'epoch': 1.77}


{'eval_loss': 1.0319130420684814, 'eval_runtime': 0.8435, 'eval_samples_per_second': 141.081, 'eval_steps_per_second': 17.783, 'epoch': 1.77}



100%|██████████| 1000/1000 [02:32<00:00,  6.58it/s]


{'train_runtime': 152.0251, 'train_samples_per_second': 26.311, 'train_steps_per_second': 6.578, 'train_loss': 0.8181923823356628, 'epoch': 1.77}


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the path where the model is saved
model_path = "./sft_output"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")


In [10]:
# Define input text for inference
input_text = "are you alive"

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate predictions
output = model.generate(
    inputs["input_ids"],
    # max_length=100,  # Adjust as needed
    # num_return_sequences=1,
    # do_sample=True,  # Enable sampling
    # top_k=50,  # Top-k sampling for diversity
    # top_p=0.95  # Nucleus sampling
)

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


Generated Text:
are you alive?

"I'm not sure. I'm not sure. I'm not


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.